In [1]:
import pywbemReq

#c = pywbemReq.WBEMConnection('https://10.141.45.135:5989', ('Local/admin', 'Password123!'), 'root/emc/smis')

c = pywbemReq.WBEMConnection('https://10.141.45.96:5989', ('Local/admin', 'Password123!'), 'root/emc/smis')

In [24]:
rep_service = c.EnumerateInstances('EMC_VNXe_ReplicationServiceLeaf')[0]
print(rep_service.tomof())

volume = c.EnumerateInstances('EMC_VNXe_StorageVolumeLeaf')[0]
print(volume.tomof())

/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/connectionpool.py:838: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/security.html
  InsecureRequestWarning)


instance of EMC_VNXe_ReplicationServiceLeaf {
	RequestedState = 12;
	Name = "ReplicationService";
	TransitioningToState = 12;
	EnabledDefault = 2;
	EnabledState = 5;
	SystemName = "FCNCH197063370";
	CreationClassName = "EMC_VNXe_ReplicationServiceLeaf";
	SystemCreationClassName = "EMC_VNXe_StorageSystemLeaf";
};



/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/connectionpool.py:838: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/security.html
  InsecureRequestWarning)


instance of EMC_VNXe_StorageVolumeLeaf {
	NumberOfBlocks = 20971520;
	RequestedState = 12;
	HealthState = 5;
	StatusDescriptions = {"OK"};
	TransitioningToState = 12;
	BlockSize = 512;
	SystemName = "FCNCH197063370";
	ExtentDiscriminator = {"SNIA:Allocated"};
	PackageRedundancy = 1;
	OtherIdentifyingInfo = {"600601608FD00A00A2663258AA2B39B5"};
	CompressionRate = 1;
	IdentifyingDescriptions = {"NAA;VPD83Type3"};
	Access = 3;
	OperationalStatus = {2};
	ParityLayout = 2;
	Usage = 2;
	ExtentStatus = {1};
	SystemCreationClassName = "EMC_VNXe_StorageSystemLeaf";
	ConsumableBlocks = 20971520;
	DeltaReservation = 0;
	CanDelete = True;
	EnabledState = 5;
	IsBasedOnUnderlyingRedundancy = True;
	Primordial = False;
	IsComposite = False;
	EMCRaidLevel = "RAID5";
	CreationClassName = "EMC_VNXe_StorageVolumeLeaf";
	DataRedundancy = 1;
	ElementName = "lun_1";
	Name = "sv_14";
	NameNamespace = 7;
	ThinlyProvisioned = True;
	NameFormat = 7;
	StorageTieringSelection = 0;
	DeviceID = "sv_14";
	NoSinglePo

In [25]:
### Create Snapshot of Volume.
result = c.InvokeMethod('CreateSynchronizationAspect',
                        rep_service.path,
                        SourceElement=volume.path,
                        SyncType=pywbemReq.Uint16(7))
print('>>> Result: ', result)

/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/connectionpool.py:838: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/security.html
  InsecureRequestWarning)


('>>> Result: ', (4096L, NocaseDict({'Job': CIMInstanceName(classname='EMC_VNXe_ConcreteJobLeaf', keybindings=NocaseDict({'InstanceID': 'N-122'}), host='10.141.45.135', namespace='root/emc/smis')})))


/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/connectionpool.py:838: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/security.html
  InsecureRequestWarning)


>>> Job affected elements:
instance of EMC_VNXe_StorageVolumeLeaf {
	NumberOfBlocks = 20971520;
	RequestedState = 12;
	HealthState = 5;
	StatusDescriptions = {"OK"};
	TransitioningToState = 12;
	BlockSize = 512;
	SystemName = "FCNCH197063370";
	ExtentDiscriminator = {"SNIA:Allocated"};
	PackageRedundancy = 1;
	OtherIdentifyingInfo = {"600601608FD00A00A2663258AA2B39B5"};
	CompressionRate = 1;
	IdentifyingDescriptions = {"NAA;VPD83Type3"};
	Access = 3;
	OperationalStatus = {2};
	ParityLayout = 2;
	Usage = 2;
	ExtentStatus = {1};
	SystemCreationClassName = "EMC_VNXe_StorageSystemLeaf";
	ConsumableBlocks = 20971520;
	DeltaReservation = 0;
	CanDelete = True;
	EnabledState = 5;
	IsBasedOnUnderlyingRedundancy = True;
	Primordial = False;
	IsComposite = False;
	EMCRaidLevel = "RAID5";
	CreationClassName = "EMC_VNXe_StorageVolumeLeaf";
	DataRedundancy = 1;
	ElementName = "lun_1";
	Name = "sv_14";
	NameNamespace = 7;
	ThinlyProvisioned = True;
	NameFormat = 7;
	StorageTieringSelection = 0;
	Devi

In [34]:
'''
job_elements = c.Associators(result[1]['Job'],
                            AssocClass='CIM_AffectedJobElement')
print('>>> Job affected elements:')
for each in job_elements:
    print(each.tomof())

created_snap = job_elements[0]
print('>>> Created snapshot: ')
print(created_snap.tomof())
'''
snaps_of_volume = c.Associators(volume.path, ResultClass='EMC_VNXe_SynchronizationAspectForSourceLeaf')
print('>>> Volume associated snapshots: ')
for each in snaps_of_volume:
    print(each.tomof())
'''
volume_of_snaps = c.Associators(created_snap.path, ResultClass='EMC_VNXe_StorageVolumeLeaf')
print('>>> Snapshot associated volumes: ')
for each in volume_of_snaps:
    print(each.tomof())

state_of_volumes = c.References(volume.path, ResultClass='CIM_SettingsDefineState')
print('Volume associated SettingsDefineState: ', len(state_of_volumes))
for each in state_of_volumes:
    print(each.tomof())
    
    
state_of_snaps = c.References(created_snap.path, ResultClass='CIM_SettingsDefineState')
print('Snapshot associated SettingsDefineState: ', len(state_of_snaps))
for each in state_of_snaps:
    print(each.tomof())
    


snapshots = c.EnumerateInstances('EMC_VNXe_SnapVolumeLeaf')
for each in snapshots:
    print(each.tomof())
'''

/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/connectionpool.py:838: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/security.html
  InsecureRequestWarning)


>>> Volume associated snapshots: 


"\nvolume_of_snaps = c.Associators(created_snap.path, ResultClass='EMC_VNXe_StorageVolumeLeaf')\nprint('>>> Snapshot associated volumes: ')\nfor each in volume_of_snaps:\n    print(each.tomof())\n\nstate_of_volumes = c.References(volume.path, ResultClass='CIM_SettingsDefineState')\nprint('Volume associated SettingsDefineState: ', len(state_of_volumes))\nfor each in state_of_volumes:\n    print(each.tomof())\n    \n    \nstate_of_snaps = c.References(created_snap.path, ResultClass='CIM_SettingsDefineState')\nprint('Snapshot associated SettingsDefineState: ', len(state_of_snaps))\nfor each in state_of_snaps:\n    print(each.tomof())\n    \n\n\nsnapshots = c.EnumerateInstances('EMC_VNXe_SnapVolumeLeaf')\nfor each in snapshots:\n    print(each.tomof())\n"

In [35]:
uis_snaps = c.EnumerateInstances('EMC_UIS_UEM_VolumeSnap', namespace='root/emc')
for each in uis_snaps:
    print(each.tomof())

/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/connectionpool.py:838: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/security.html
  InsecureRequestWarning)


instance of C4CB_UIS_UEM_VolumeSnap {
	SourceLunId = "root/emc:EMC_UEM_StorageVolumeLeaf%InstanceID=sv_14";
	EMCAllowAttach = True;
	Description = "";
	AutoDelete = True;
	CreatorUserId = "root/emc:C4CB_UIS_User%InstanceID=admin";
	InstanceID = "root/emc:EMC_UEM_VolumeSnapLeaf%InstanceID=38654705724";
	OperationalStatus = {2};
	IsModified = False;
	FriendlyID = "38654705724";
	IsSystemSnap = False;
	ReplicationStatus = 0;
	CreatorType = 2;
	SourceSnapId = "root/emc:EMC_UEM_VolumeSnapLeaf%InstanceID=";
	SourceStorageResourceId = "root/emc:EMC_UEM_StorageVolumeLeaf%InstanceID=sv_14";
	InstallDate = 20161121032359.800130+000;
	Size = 10737418240;
	Name = "LUNSNAP_20161121T032356";
};

instance of C4CB_UIS_UEM_VolumeSnap {
	SourceLunId = "root/emc:EMC_UEM_StorageVolumeLeaf%InstanceID=sv_14";
	EMCAllowAttach = True;
	Description = "";
	AutoDelete = True;
	CreatorUserId = "";
	InstanceID = "root/emc:EMC_UEM_VolumeSnapLeaf%InstanceID=38654705725";
	OperationalStatus = {2};
	IsModified = False

In [4]:
uis_snaps = c.EnumerateInstances('EMC_UIS_UEM_VolumeSnap', namespace='root/emc')
for each in uis_snaps:
    print(each.properties)

/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/connectionpool.py:838: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/security.html
  InsecureRequestWarning)


NocaseDict({'SourceLunId': CIMProperty(name='SourceLunId', value='root/emc:EMC_UEM_StorageVolumeLeaf%InstanceID=sv_21', type='string', class_origin=None, array_size=None, propagated=None, is_array=False, reference_class=None, qualifiers=NocaseDict({}), embedded_object=None), 'EMCAllowAttach': CIMProperty(name='EMCAllowAttach', value=True, type='boolean', class_origin=None, array_size=None, propagated=None, is_array=False, reference_class=None, qualifiers=NocaseDict({}), embedded_object=None), 'IsPromoted': CIMProperty(name='IsPromoted', value=True, type='boolean', class_origin=None, array_size=None, propagated=None, is_array=False, reference_class=None, qualifiers=NocaseDict({}), embedded_object=None), 'PromotedLUNId': CIMProperty(name='PromotedLUNId', value=19L, type='uint32', class_origin=None, array_size=None, propagated=None, is_array=False, reference_class=None, qualifiers=NocaseDict({}), embedded_object=None), 'Description': CIMProperty(name='Description', value='', type='string'